In [75]:
%run Requirements.ipynb

24/06/04 14:50:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [76]:
df = spark.sql(
    """
    SELECT DISTINCT SM.Artists
    FROM PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/AppleMusic` AS AM
    INNER JOIN PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/SpotifyMusic` AS SM 
    ON LTRIM(RTRIM(LOWER(AM.`Artist Name`))) = LTRIM(RTRIM(LOWER(SM.Artists)))
    LEFT JOIN PARQUET.`/Users/ronitguptaaa/Documents/Music-Fusion-APP/Staging/YoutubeMusic` AS YM
    ON LTRIM(RTRIM(LOWER(AM.`Artist Name`))) = LTRIM(RTRIM(LOWER(YM.`Artist Name`)))
    """
)

24/06/04 14:50:13 WARN ObjectStore: Failed to get database parquet, returning NoSuchObjectException
24/06/04 14:50:13 WARN ObjectStore: Failed to get database parquet, returning NoSuchObjectException
24/06/04 14:50:13 WARN ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [77]:
artists_df = df.select(F.col('Artists')).distinct()
artists_list = [row['Artists'] for row in artists_df.collect()]

news_api_key = News_API.get('API_KEY')

news_data = []
for artist in artists_list:
    url = f"https://newsapi.org/v2/everything?q={artist}&pageSize=10&apiKey={news_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json()['articles']
        for article in articles:
            news_data.append({
                'Artist': artist,
                'Title': article.get('title'),
                'Description': article.get('description'),
                'URL': article.get('url'),
                'Content': article.get('content')
            })
    else:
        print(f"Failed to fetch news for artist: {artist}, Status Code: {response.status_code}")

news_df = pd.DataFrame(news_data)
news_spark_df = spark.createDataFrame(news_df)

news_spark_df.write.mode('overwrite').format('parquet').save('Mart/MusicFusion-Details')
news_spark_df.write.mode('overwrite').format('parquet').saveAsTable('MusicFusionDetails')

24/06/04 14:50:22 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
